In [1]:
from fastai.text import *
import html

BOS = 'xbos'  # beginning-of-sentence tag
FLD = 'xfld'  # data field tag

re1 = re.compile(r'  +')
LM_PATH = Path('./LM')

## adjust data set to fit in memory

In [2]:
df = pd.read_csv(LM_PATH / 'train.csv', header=None)
print(df.shape)
df = df[:int(df.shape[0]*.1)]
print(df.shape)
df.to_csv(LM_PATH / 'train.csv', header=False, index=False)

(547665, 1)
(54766, 1)


In [3]:
df = pd.read_csv(LM_PATH / 'val.csv', header=None)
print(df.shape)
df = df[:int(df.shape[0]*.1)]
print(df.shape)
df.to_csv(LM_PATH / 'val.csv', header=False, index=False)

(136917, 1)
(13691, 1)


## tokenize

In [4]:
def fixup(x):
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>','u_n').replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))


def get_texts(df, n_lbls, lang='es'):
    if len(df.columns) == 1:
        labels = []
        texts = f'\n{BOS} {FLD} 1 ' + df[0].astype(str)
    else:
        labels = df.iloc[:,range(n_lbls)].values.astype(np.int64)
        texts = f'\n{BOS} {FLD} 1 ' + df[n_lbls].astype(str)
        for i in range(n_lbls+1, len(df.columns)): texts += f' {FLD} {i-n_lbls+1} ' + df[i].astype(str)
    texts = list(texts.apply(fixup).values)

    tok = Tokenizer(lang=lang).proc_all_mp(partition_by_cores(texts), lang=lang)
    return tok, list(labels)


def get_all(df, n_lbls, lang='es'):
    tok, labels = [], []
    for i, r in enumerate(df):
        print(i)
        tok_, labels_ = get_texts(r, n_lbls, lang=lang)
        tok += tok_
        labels += labels_
    return tok, labels

In [5]:
df = pd.read_csv(LM_PATH / 'train.csv', header=None, chunksize=5500)
tok, _ = get_all(df, 0, lang='es')
np.save('tmp/tok_trn.npy', tok)
del df, tok
print('saved')

0
1
2
3
4
5
6
7
8
9
saved


In [5]:
df = pd.read_csv(LM_PATH / 'val.csv', header=None, chunksize=5500)
tok, _ = get_all(df, 0, lang='es')
np.save('tmp/tok_val.npy', tok)
print('saved')

0
1
2
saved
